### Assignment 3 - Voting Rules in Python

**First, we read the .csv file**

In [8]:
import csv
import math

def read_file(filename):
    with open(filename, newline='') as file:
        reader = csv.reader(file, delimiter=';')
        next(reader)  # skip the first line (headers)
        data = list(reader) # convert .csv object to a list of lists.
    return data

# read_file("preferences_assignment3.csv")

**1. Compute a function Plurality returning the result of a plurality voting.**

* The candidate with most votes wins, even if it lacks majority support.


In [2]:
def Plurality(preferences_filename, rounds = 1): # we pass as an argument the filename with the preferences
    preferences = read_file(preferences_filename) # it will return a list of lists containing the lines of the .csv file.
    # each list will be given as: [no.Voters, preference1, preference2,...]
    votes = {} # stores all votes corresponding to each candidate: candidate: votes
    for row in preferences:
        if row[1] not in votes.keys():
            votes[row[1]] = int(row[0])
        else:
            votes[row[1]] += int(row[0])
            
    sorted_votes = sorted(votes.items(), key=lambda x: x[1], reverse=True) # returns a list of tuples (candidate, votes) ordered by number of votes.
    if rounds == 1: # just one round, return most voted candidate.
        candidate_elected = sorted_votes[0] # gives the tuple (candidate, votes) corresponding to candidate who received most votes
        print(f"\nCandidate '{candidate_elected[0]}' is the plurality winner. It's elected with {candidate_elected[1]} votes.")
        
    elif rounds == 2: # have to perform a second round of plurality.
        abs_majority = math.ceil((sum(votes.values()))/2) # absolute majority.
        top2_candidates = sorted_votes[:2] # 2 candidates with most votes.
        if top2_candidates[0][1] >= abs_majority: # if a candidate receives more than 50% of votes, then it is elected.
            candidate_elected = top2_candidates[0]
            print(f"\nCandidate '{candidate_elected[0]}' is the plurality with runoff winner. It's elected with {candidate_elected[1]} votes.")
        else: # we go to the second round.
            candidate1 = top2_candidates[0][0]
            candidate2 = top2_candidates[1][0]
            for row in preferences: # for each possible preference
                if row[1] != candidate1 and row[1] != candidate2: # skip preferences where either candidate1 or candidate2 are preferred among the rest of candidates.
                    candidates_ranking = row[2:] # get the preference order of candidates skipping the first candidate.
                    if candidates_ranking.index(candidate1) < candidates_ranking.index(candidate2): # candidate1 beats candidate2
                        votes[candidate1] += int(row[0])
                    else: # candidate2 beats candidate1
                        votes[candidate2] += int(row[0])
            sorted_votes = sorted(votes.items(), key=lambda x: x[1], reverse=True) # returns a list of tuples (candidate, votes) ordered by number of votes.
            candidate_elected = sorted_votes[0]
            print(f"\nCandidate '{candidate_elected[0]}' is the plurality with runoff winner. It's elected with {candidate_elected[1]} votes.")
    return None



# Calling the function:
Plurality("preferences_assignment3.csv") # to perform plurality (1 round).


Candidate 'd' is the plurality winner. It's elected with 10 votes.


**2. Compute a function PluralityRunoff returning the result of a plurality Runoff voting (plurality with two
rounds).**

* First round: apply plurality.
    * If candidate with most votes receives more than 50% of total votes -> elected, otherwise, go to round 2 with 2 most-voted candidates.

* Second round: keep 2 most-voted candidates and apply plurality.

In [3]:
def PluralityRunoff(preferences_filename):
    Plurality(preferences_filename, 2) # pluralityRunoff is plurality with 2 rounds.



# Calling the function:
PluralityRunoff("preferences_assignment3.csv") # to perform pluralityRunoff.


Candidate 'a' is the plurality with runoff winner. It's elected with 17 votes.


**3. Compute a function CondorcetVoting returning the result of the application of the Condorcet principle (the existence of the Condorcet winner).**

* For a candidate to be elected (for a Condorcet winner to exist), it must beat all rest of candidates.

* Can happen that there is no Condorcet winner -> Condorcet's paradox.

In [4]:
def CondorcetVoting(preferences_filename):
    preferences = read_file(preferences_filename) # it will return a list of lists containing the lines of the .csv file.
    candidates = sorted(preferences[0][1:]) # get the candidates in a list.
    # for a candidate to be elected, it must beat all other candidates.
    condorcet_winner = ""
    for c1 in candidates:
        winner = True
        for c2 in candidates:
            if c1 == c2:
                continue # go to next candidate c2.
            c1_beats_c2 = 0 # no. of votes c1 beating c2.
            c2_beats_c1 = 0 # no. of votes c2 beating c1.
            for row in preferences:
                votes = int(row[0]) # get the number of votes for current preference.
                candidates_ranking = row[1:] # get the preference order of candidates.
                if candidates_ranking.index(c1) < candidates_ranking.index(c2): # c1 beats c2
                    c1_beats_c2 +=votes
                else: # c2 beats c1 
                    c2_beats_c1 += votes
            if c1_beats_c2 <= c2_beats_c1: # if c1 does not beat c2, then, for sure it cannot be the condorcet winner.
                winner = False
                break # exit c2 loop -> go to next c1 candidate as this one is not the condorcet winner.
        if winner: # if candidate c1 beats all other candidates, then it is the condorcet winner.
            condorcet_winner = c1
    if condorcet_winner == "": # no winner.
        print("\nThere is no Condorcet winner.") # no candidate beats all other candidates -> Condorcet's Paradox.
    else: # condorcet winner exists.
        print(f"\nCandidate '{condorcet_winner}' is the Condorcet winner. It's elected since it beats all the rest of candidates.")
    return None



# Calling the function:
CondorcetVoting("preferences_assignment3.csv")



Candidate 'c' is the Condorcet winner. It's elected since it beats all the rest of candidates.


**4. Compute a function BordaVoting returning the result of the application of the Borda principle.**

* Choose candidate whose Borda's score (based on ranking) is the smallest.


In [5]:
def BordaVoting(preferences_filename): # Borda winner corresponds to the candidate with miminum rank score.
    preferences = read_file(preferences_filename) # it will return a list of lists containing the lines of the .csv file.
    candidates = sorted(preferences[0][1:]) # get the candidates in a list.
    borda_scores = {c: 0 for c in candidates} # to store each candidate score using the Borda procedure.
    borda_winner = ""
    for c in candidates:
        for row in preferences:
            index_c = row.index(c) # obtain the ranking position of candidate c.
            borda_scores[c] += int(row[0]) * index_c # multiply no. votes times the ranking position.
    borda_winner = min(borda_scores, key = borda_scores.get) # borda winner -> candidate with minimum score.
    print(f"\nCandidate '{borda_winner}' is the Borda winner. Its corresponding Borda score is {borda_scores[borda_winner]}.")
    return None



# Calling the function:
BordaVoting("preferences_assignment3.csv")


Candidate 'b' is the Borda winner. Its corresponding Borda score is 60.


**5. Is it possible to elaborate an election example with n ≥ 60 and m ≥ 8 where the winner is the same for the four
voting rules Plurality, Plurality with Runoff, Condorcet Principle and Borda rules ?**

*In your example, **no more than 50%** of voters has the **same “best candidate”** and **no more than 40%** of voters has the
**same “worst candidate”**. You should implement, separately, a python function allowing to test if these two conditions
are satisfied.*

For a winner candidate to be the same in the four voting rules, it must satisfy:

* The candidate should have the most "first-place" votes to be elected by Plurality principle.

* Should be the 2 most voted candidates to win the Plurality with runoff and at the same time, the preferences for which it is not preferred, should be in the top positions.

* Should beat all rest of candidates, to be the Condorcet winner.

* Should have the smallest Borda score.


In [6]:
# n >= 60 (min. 60 voters)
# m >= 8 (min. 8 candidates)


# Function to show the best candidate condition is satisfied: no more than 50% of voters have the same "best candidate"
# AND the worst candidate condition is satisfied: no more than 40% of voters have the same "worst candidate".
def best_worst_candidate_condition(preferences_filename):
    preferences = read_file(preferences_filename) # it will return a list of lists containing the lines of the .csv file.
    total_voters = sum(int(row[0]) for row in preferences) # we sum all voters.
    candidates = sorted(preferences[0][1:]) # get the candidates in a list.
    first_candidates_counts = {c: 0 for c in candidates} # dict to count the number of votes each candidate has being ranked the 1st.
    last_candidates_counts = {c: 0 for c in candidates} # dict to count the number of votes each candidate has being ranked the last.
    for row in preferences:
        votes = int(row[0])
        first_candidate = row[1] # obtain first candidate in current preference.
        last_candidate = row[-1] # obtain last candidate in current preference.
        first_candidates_counts[first_candidate] += votes
        last_candidates_counts[last_candidate] += votes
    best_candidate_count = max(first_candidates_counts.values())
    worst_candidate_count = max(last_candidates_counts.values())
    # print(best_candidate_count)
    # print(worst_candidate_count)
    if (best_candidate_count <= 0.5 * total_voters) and (worst_candidate_count <= 0.4 * total_voters):
        print("\nIn the example, both requested conditions are satisfied: no more than 50% of voters have the same 'best candidate' and no more than 40% have the same 'worst candidate'.")
    else:
        print("The example does not satisfy the requested conditions.")
    return None



# "example_exercise5_assignment3.csv" -> 70 voters and 9 candidates (a to i).
# Calling function:
best_worst_candidate_condition("example_exercise5_assignment3.csv")
Plurality("example_exercise5_assignment3.csv")
PluralityRunoff("example_exercise5_assignment3.csv")
CondorcetVoting("example_exercise5_assignment3.csv")
BordaVoting("example_exercise5_assignment3.csv")



In the example, both requested conditions are satisfied: no more than 50% of voters have the same 'best candidate' and no more than 40% have the same 'worst candidate'.

Candidate 'a' is the plurality winner. It's elected with 35 votes.

Candidate 'a' is the plurality with runoff winner. It's elected with 35 votes.

Candidate 'a' is the Condorcet winner. It's elected since it beats all the rest of candidates.

Candidate 'a' is the Borda winner. Its corresponding Borda score is 119.


**6. Is it possible to elaborate an election example with n ≥ 60 and m ≥ 8 where the unique winner is not the same
for the four voting rules Plurality, Plurality with Runoff, Condorcet Principle and Borda rules (we should have 4
different winners) ?**

*In your example, **no more than 50%** of voters has the **same “best candidate”** and **no more than 40%** of voters has the
**same “worst candidate”**. In order to test if these two conditions are satisfied, you can use the function implemented
above, in question 5.*

In [7]:
# "example_exercise6_assignment3.csv" -> 70 voters and 9 candidates (a to i).
# Calling function:
best_worst_candidate_condition("example_exercise6_assignment3.csv")
Plurality("example_exercise6_assignment3.csv")
PluralityRunoff("example_exercise6_assignment3.csv")
CondorcetVoting("example_exercise6_assignment3.csv")
BordaVoting("example_exercise6_assignment3.csv")


In the example, both requested conditions are satisfied: no more than 50% of voters have the same 'best candidate' and no more than 40% have the same 'worst candidate'.

Candidate 'a' is the plurality winner. It's elected with 25 votes.

Candidate 'd' is the plurality with runoff winner. It's elected with 45 votes.

Candidate 'b' is the Condorcet winner. It's elected since it beats all the rest of candidates.

Candidate 'c' is the Borda winner. Its corresponding Borda score is 205.
